In [1]:
# # Requerimientos de librerías:
# !pip install spacy
# !python -m spacy download es_core_news_md

# !pip install transformers
# !pip install transformers scipy ftfy accelerate

# !git clone https://github.com/JorgeSauri/COMSINT.git 

In [1]:
from recomendaciones_comsint import Recomendador

c:\Users\jsaur\anaconda3\envs\ENV1\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
recomendador128 = Recomendador(fuente='recipes_traducido.csv')
recomendador128.CargarModelo(emb_size=128, version=4)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Modelo Modelos/Modelo_Nut_FV_DistilBERT_04_EMBED-128_CNN.h5 cargado con éxito.
Modelo Modelos/Modelo_Precios_FV_DistilBERT_04_EMBED-128_CNN.h5 cargado con éxito.


In [ ]:
import numpy as np

nlp = recomendador128.nlp
tokens1 = [nlp('aceite de oliva'), nlp('pepino'), nlp('apio'), nlp('sal')]
tokens2 = [nlp('frijol'), nlp('arroz'), nlp('huevo'), nlp('sal'), nlp('azucar')]
tokens3 = [nlp('vainilla'), nlp('almendras'), nlp('coco')]
receta = ' tazas de fechas de medjool picadas, más o menos picadas, 2 tazas de anacardos crudos, 1 taza de almendras crudas o asadas sin sal, ¾ taza de cacao de alta calidad sin azúcar en polvo (como Guittard® Cocoa Rouge), 2 cucharadas de aceite de coco, derretido, ½ taza sin azúcar, por celo, cedido por cable. Coco, 2 cucharaditas de extracto de vainilla, 1 cucharada de espresso frío, o más según sea necesario (o agua), ½ cucharadita de sal kosher, ½ pimienta de cayena'
tokenReceta = nlp(receta)

tokensComp = tokens2
umbral = 0.2
sim = 0
for t1 in tokensComp:
    print(tokenReceta.similarity(t1))
    if tokenReceta.similarity(t1)>=umbral: 
        sim += tokenReceta.similarity(t1)
    if sim>1: sim = sim - 1

sim, sim/len(tokensComp)

In [11]:
recomendador128.FiltrarRecetario_por_CanastaBasica(similitud=0.8, verbose=False, max_rows=5)
df_platillos_recomendados2 = recomendador128.Calcular_InfoNutricional(verbose=False)

ingredientes recibidos: 
Buscando recetas con ingredientes de la canasta básica... 



100%|██████████| 1090/1090 [00:09<00:00, 115.46it/s]


 

 5 platillos encontrados con similitud mayor a 0.8
1/1 [==============================] - 0s 150ms/step


In [12]:
df2 = df_platillos_recomendados2.sort_values(by=['similitud'], ascending=False)
df2

,nombre_del_platillo,ingredientes,similitud,kcal,proteinas_gr,carbohidratos_gr,grasas_gr,puntaje_platillo
763,Ensalada de frutas de papaya de mango,"¼ taza de jugo de lima recién exprimido, ¼ de ...",0.967,220.58,10.97,23.19,7.58,0.3
278,Fuego de sandía y salsa de hielo,"3 tazas de sandía picada, ½ taza de pimiento v...",0.967,326.34,15.74,34.13,11.29,0.3
501,Sopa de calabaza ahumada butternut,1 calabaza de butternut: pelado y cortado en c...,0.967,4036.46,166.58,334.08,219.47,0.7
634,Char Siu (carne de cerdo de barbacoa china),"2 cucharadas de miel, 2 cucharadas de vino de ...",0.967,7290.12,343.50,664.32,363.86,0.7
647,Alitas de pollo de mango-hábana,"30 secciones de ala de pollo, 1 (12 onzas) pue...",0.967,7972.26,413.11,776.46,363.91,0.3


In [13]:
df3 = df2[df2['puntaje_platillo']>=0.5]
df3

,nombre_del_platillo,ingredientes,similitud,kcal,proteinas_gr,carbohidratos_gr,grasas_gr,puntaje_platillo
501,Sopa de calabaza ahumada butternut,1 calabaza de butternut: pelado y cortado en c...,0.967,4036.46,166.58,334.08,219.47,0.7
634,Char Siu (carne de cerdo de barbacoa china),"2 cucharadas de miel, 2 cucharadas de vino de ...",0.967,7290.12,343.50,664.32,363.86,0.7


In [14]:
df4 = recomendador128.Calcular_Precios(df3, verbose=False)
df4

1/1 [==============================] - 0s 143ms/step


c:\Users\jsaur\Dropbox\UNIR MAESTRIA\MAESTRIA IA\SEMINARIO DE INNOVACION\repo\COMSINT\recomendaciones_comsint.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfFiltrados['costo_receta'] = Precios


,nombre_del_platillo,ingredientes,similitud,kcal,proteinas_gr,carbohidratos_gr,grasas_gr,puntaje_platillo,costo_receta
501,Sopa de calabaza ahumada butternut,1 calabaza de butternut: pelado y cortado en c...,0.967,4036.46,166.58,334.08,219.47,0.7,620.79
634,Char Siu (carne de cerdo de barbacoa china),"2 cucharadas de miel, 2 cucharadas de vino de ...",0.967,7290.12,343.50,664.32,363.86,0.7,123.23


In [17]:
df4[df4['costo_receta']>=50].sort_values(by=['costo_receta','kcal', 'similitud'], ascending=True)

,nombre_del_platillo,ingredientes,similitud,kcal,proteinas_gr,carbohidratos_gr,grasas_gr,puntaje_platillo,costo_receta
634,Char Siu (carne de cerdo de barbacoa china),"2 cucharadas de miel, 2 cucharadas de vino de ...",0.967,7290.12,343.50,664.32,363.86,0.7,123.23
501,Sopa de calabaza ahumada butternut,1 calabaza de butternut: pelado y cortado en c...,0.967,4036.46,166.58,334.08,219.47,0.7,620.79


In [39]:
resultados = df4.to_dict(orient='records')

In [44]:
resultados[0]

{'nombre_del_platillo': 'Guiso de filete a la mexicana',
 'ingredientes': '3 unidades de Jitomates maduros picados en cubos\n 5 unidades de Tomates verdes picados en cubos\n 1 unidad de Pimiento morrón verde picado en cubos\n 1 unidad de Chile serrano verde picado\n ½ unidad de Cebolla picada en cubos\n ¼ unidad de Cebolla\n 1 diente de Ajo picado\n ½ manojo de Cilantro pequeño\n 1 pizca de Sal o consomé de pollo en polvo\n 1 pizca de Pimienta molida\n 5 filetes de Milanesa o filetes de res picada en cuadros pequeños',
 'similitud': 0.6,
 'kcal': 9780.98,
 'proteinas_gr': 460.53,
 'carbohidratos_gr': 991.73,
 'grasas_gr': 427.63,
 'puntaje_platillo': 0.3,
 'costo_receta': 391.27}